In [ ]:
import numpy as np
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import math
import cv2
import torch
import pandas as pd
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor
import torchvision.models as models

In [ ]:
import os
import zipfile
local_zip = '/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Animals.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/dataset')
zip_ref.close()

In [ ]:
# 32x32x4 비트 구성

class RGB_32x32_4_Dataset(torch.utils.data.Dataset):
    def __init__(self, annotation_path, root_dir='/dataset'):
        'Initialization'
        self.data_annotation = pd.read_csv(os.path.join(annotation_path))
        self.data_path = self.data_annotation['filepath']
        self.labels = self.data_annotation['label']
        self.root_dir = root_dir
        self.transforms = Compose([
            ToTensor()
        ])

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_path)

    def _preprocessing(self, image):
        # Resize to 32x32
        img_resized = cv2.resize(image, (32, 32), interpolation=cv2.INTER_LANCZOS4)
        # 4bit를 RGB에 분배 (2-1-1 bit: R: 2 bits, G: 1 bit, B: 1 bit)
        img_array = np.array(img_resized, dtype=np.float32)  # 0~255
        r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
        r_2bit = np.floor((r / 255) * 1).astype(np.uint8)  # 0~3
        g_1bit = np.floor((g / 255) * 1).astype(np.uint8)  # 0~1
        b_1bit = np.floor((b / 255) * 2).astype(np.uint8)  # 0~1
        # Simulate RGB output
        img_rgb = np.zeros_like(img_resized, dtype=np.uint8)
        img_rgb[:, :, 0] = (r_2bit / 1) * 255
        img_rgb[:, :, 1] = (g_1bit / 1) * 255
        img_rgb[:, :, 2] = (b_1bit / 2) * 255
        img_pil = Image.fromarray(img_rgb, mode='RGB')
        return img_pil

    def __getitem__(self, index):
        'Generates one sample of data'
        file_path = os.path.join(self.root_dir, self.data_path[index])
        input_image = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        processed_image = self._preprocessing(input_image)
        X = self.transforms(processed_image)  # Convert to tensor
        y = torch.tensor(self.labels[index]).long()
        return X, y  # X: 3x32x32 tensor (4-bit RGB), y: label

In [ ]:
import random
## make by using custom dataset class
RGB_32x32_4_trainset = RGB_32x32_4_Dataset(annotation_path = '/dataset/train_annotation.csv')
RGB_32x32_4_testset = RGB_32x32_4_Dataset(annotation_path = '/dataset/test_annotation.csv')

print('total training images:', len(RGB_32x32_4_trainset))
print('total test images:', len(RGB_32x32_4_testset))
print('Torch size:', RGB_32x32_4_trainset[0][0].shape)
print('rgb 이미지 분류 데이터 확인')

## visualize
class_names=['Cat','Dog','Tiger','Zebra']
for i in range(9):
  random_index = random.randint(0,8000)
  image, label = RGB_32x32_4_trainset[random_index]
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image.permute(1,2,0).numpy())
  plt.title(class_names[int(label)])
  #plt.axis("off")

In [ ]:
# 25x27x6
class RGB_25x27_6_Dataset(torch.utils.data.Dataset):
    def __init__(self, annotation_path, root_dir='/dataset'):
        'Initialization'
        self.data_annotation = pd.read_csv(os.path.join(annotation_path))
        self.data_path = self.data_annotation['filepath']
        self.labels = self.data_annotation['label']
        self.root_dir = root_dir
        self.transforms = Compose([
            ToTensor()
        ])

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_path)

    def _preprocessing(self, image):
        # Resize to 25x27
        img_resized = cv2.resize(image, (25, 27), interpolation=cv2.INTER_LANCZOS4)
        # 양자화 6-bit RGB (2-2-2 bit: R: 2 bits, G: 2 bits, B: 2 bits)
        img_array = np.array(img_resized, dtype=np.float32)  # 0~255
        r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
        r_2bit = np.floor((r / 255) * 3).astype(np.uint8)  # 0~3 (2 bits)
        g_2bit = np.floor((g / 255) * 3).astype(np.uint8)  # 0~3 (2 bits)
        b_2bit = np.floor((b / 255) * 3).astype(np.uint8)  # 0~3 (2 bits)
        # Simulate RGB output for CNN
        img_rgb = np.zeros_like(img_resized, dtype=np.uint8)
        img_rgb[:, :, 0] = (r_2bit / 3) * 255  # 0, 85, 170, 255
        img_rgb[:, :, 1] = (g_2bit / 3) * 255  # 0, 85, 170, 255
        img_rgb[:, :, 2] = (b_2bit / 3) * 255  # 0, 85, 170, 255
        img_pil = Image.fromarray(img_rgb, mode='RGB')
        return img_pil

    def __getitem__(self, index):
        'Generates one sample of data'
        file_path = os.path.join(self.root_dir, self.data_path[index])
        input_image = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        processed_image = self._preprocessing(input_image)
        X = self.transforms(processed_image)  # Convert to tensor
        y = torch.tensor(self.labels[index]).long()
        return X, y  # X: 3x27x25 tensor (6-bit RGB), y: label

In [ ]:
## make by using custom dataset class
RGB_25x27_6_trainset = RGB_25x27_6_Dataset(annotation_path = '/dataset/train_annotation.csv')
RGB_25x27_6_testset = RGB_25x27_6_Dataset(annotation_path = '/dataset/test_annotation.csv')

print('total training images:', len(RGB_25x27_6_trainset))
print('total test images:', len(RGB_25x27_6_testset))
print('Torch size:', RGB_25x27_6_trainset[0][0].shape)
print('rgb 이미지 분류 데이터 확인')

## visualize
class_names=['Cat','Dog','Tiger','Zebra']
for i in range(9):
  random_index = random.randint(0,8000)
  image, label = RGB_25x27_6_trainset[random_index]
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image.permute(1,2,0).numpy())
  plt.title(class_names[int(label)])
  #plt.axis("off")

In [ ]:
# 36x37_1비트 구성

class RGB_36x37_1_Dataset(torch.utils.data.Dataset):
    def __init__(self, annotation_path, root_dir='/dataset'):
        'Initialization'
        self.data_annotation = pd.read_csv(os.path.join(annotation_path))
        self.data_path = self.data_annotation['filepath']
        self.labels = self.data_annotation['label']
        self.root_dir = root_dir
        self.transforms = Compose([
            ToTensor()
        ])

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_path)

    def _preprocessing(self, image):
        # Resize to 36x37
        img_resized = cv2.resize(image, (36, 37), interpolation=cv2.INTER_LANCZOS4)
        # Quantize to 3-bit RGB (1-1-1 bit: R: 1 bit, G: 1 bit, B: 1 bit)
        img_array = np.array(img_resized, dtype=np.float32)  # 0~255
        r, g, b = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
        r_1bit = np.floor((r / 255) * 1).astype(np.uint8)  # 0~1 (1 bit)
        g_1bit = np.floor((g / 255) * 1).astype(np.uint8)  # 0~1 (1 bit)
        b_1bit = np.floor((b / 255) * 1).astype(np.uint8)  # 0~1 (1 bit)
        # Simulate RGB output for CNN
        img_rgb = np.zeros_like(img_resized, dtype=np.uint8)
        img_rgb[:, :, 0] = (r_1bit / 1) * 255  # 0, 255
        img_rgb[:, :, 1] = (g_1bit / 1) * 255  # 0, 255
        img_rgb[:, :, 2] = (b_1bit / 1) * 255  # 0, 255
        img_pil = Image.fromarray(img_rgb, mode='RGB')
        return img_pil

    def __getitem__(self, index):
        'Generates one sample of data'
        file_path = os.path.join(self.root_dir, self.data_path[index])
        input_image = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
        processed_image = self._preprocessing(input_image)
        X = self.transforms(processed_image)  # Convert to tensor
        y = torch.tensor(self.labels[index]).long()
        return X, y  # X: 3x37x36 tensor (3-bit RGB), y: label

In [ ]:
## make by using custom dataset class
RGB_36x37_1_trainset = RGB_36x37_1_Dataset(annotation_path = '/dataset/train_annotation.csv')
RGB_36x37_1_testset = RGB_36x37_1_Dataset(annotation_path = '/dataset/test_annotation.csv')

print('total training images:', len(RGB_36x37_1_trainset))
print('total test images:', len(RGB_36x37_1_testset))
print('Torch size:', RGB_36x37_1_trainset[0][0].shape)
print('rgb 이미지 분류 데이터 확인')

## visualize
class_names=['Cat','Dog','Tiger','Zebra']
for i in range(9):
  random_index = random.randint(0,8000)
  image, label = RGB_36x37_1_trainset[random_index]
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image.permute(1,2,0).numpy())
  plt.title(class_names[int(label)])
  #plt.axis("off")

In [ ]:
## make model and using GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: ', device)

In [ ]:
import multiprocessing

num_cpus = multiprocessing.cpu_count()
print(f"Number of available CPUs: {num_cpus}")


In [ ]:
!pip install torchinfo -qq

In [ ]:
from torchsummary import summary

In [ ]:
# 수정된 BasicLeNet
import torch.nn as nn
class LeNetCustom(nn.Module):
    def __init__(self):
        super(LeNetCustom, self).__init__()
      # Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # 채널 감소
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # 스트라이드 2
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.shortcut = nn.Conv2d(32, 64, kernel_size=1, stride=2, padding=0)  # 잔차 연결
        # Block 2
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)  # 128x16x16 -> 128x1x1
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128, 4)  # 4개 클래스, FC 레이어 간소화
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # Block 1
        x1 = self.relu1(self.bn1(self.conv1(x)))  # 32x64x64
        shortcut = self.shortcut(x1)  # 64x32x32
        x2 = self.relu2(self.bn2(self.conv2(x1)))  # 64x32x32
        x2 = x2 + shortcut  # 잔차 연결
        # Block 2
        x3 = self.relu3(self.bn3(self.conv3(x2)))  # 128x16x16
        # Global Average Pooling
        x = self.gap(x3)  # 128x1x1
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

model1 = LeNetCustom().to(device)
summary(model1, (3,32, 32))

In [ ]:
class AlexNetCustum(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2), # 96→32, 11→5
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),          # 192→64, 5→3
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),          # 256→64
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            # Fully connected 파라미터 대폭 감소
            # 마지막 feature map을 1x1로 줄임
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*1*1, 8),
            nn.ReLU(inplace=True),
            nn.Linear(8, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model2 = AlexNetCustum(num_classes=4).to(device)
summary(model2,(3,32,32))

In [ ]:
class VGGCustom(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling 추가
        )
        self.classifier = nn.Sequential(
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

model3 = VGGCustom(num_classes=4).to(device)
summary(model3,(3,32,32))

In [ ]:
## parameters
epoch = 40
batchsize = 32

## RGB_32x32_4_dataloader
RGB_32x32_4_train_loader = torch.utils.data.DataLoader(RGB_32x32_4_trainset,
                                          batch_size=batchsize,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

RGB_32x32_4_test_loader = torch.utils.data.DataLoader(RGB_32x32_4_testset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

## RGB_25x27_6_dataloader
RGB_25x27_6_train_loader = torch.utils.data.DataLoader(RGB_25x27_6_trainset,
                                          batch_size=batchsize,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

RGB_25x27_6_test_loader = torch.utils.data.DataLoader(RGB_25x27_6_testset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

## RGB_36x37_1_dataloader
RGB_36x37_1_train_loader = torch.utils.data.DataLoader(RGB_36x37_1_trainset,
                                          batch_size=batchsize,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

RGB_36x37_1_test_loader = torch.utils.data.DataLoader(RGB_36x37_1_testset,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

## loss function
criterion = torch.nn.CrossEntropyLoss() # Cross Entropy

## optimizer setting
optimizer1 = torch.optim.Adam(model1.parameters(), ## Adam optimizer
                            lr=0.001)
lr1  = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=5, gamma=0.5)  # 5에폭마다 lr 0.5배

optimizer2 = torch.optim.Adam(model2.parameters(), ## Adam optimizer
                            lr=0.001)
lr2  = torch.optim.lr_scheduler.StepLR(optimizer2, step_size=5, gamma=0.5)  # 5에폭마다 lr 0.5배

optimizer3 = torch.optim.Adam(model3.parameters(), ## Adam optimizer
                            lr=0.001)
lr3  = torch.optim.lr_scheduler.StepLR(optimizer3, step_size=5, gamma=0.5)  # 5에폭마다 lr 0.5배



In [ ]:
# model training function
def train(model, optimizer, train_loader, epoch):
  train_loss = []
  train_accuracy = []
  avg_loss = 0
  avg_accuracy = 0
  model.train()

  for i, (X,y) in enumerate(train_loader):
      X,y = X.to(device), y.to(device)
      X=X.float()
      optimizer.zero_grad()
      predict = model(X)
      loss = criterion(predict, y)
      loss.backward()
      optimizer.step()

      _, predicted_classes = torch.max(predict, 1) # Get the predicted class index
      correct_predictions = (predicted_classes == y).sum().item()
      accuracy = correct_predictions / X.shape[0]

      train_accuracy.append(accuracy)
      train_loss.append(loss.item())

  avg_loss = sum(train_loss) /len(train_loss)
  avg_accuracy = sum(train_accuracy) / len(train_accuracy)

  print(f'epoch {epoch}) train loss : {avg_loss:.4f} / train_accuracy : {avg_accuracy:.4f}')
  return avg_loss, avg_accuracy

In [ ]:
def test(model, test_loader):
    model.eval()
    test_loss = []
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            X=X.float()
            predict = model(X)
            loss = criterion(predict, y)
            test_loss.append(loss.item())

            _, predicted_classes = torch.max(predict, 1)
            correct_predictions += (predicted_classes == y).sum().item()
            total_samples += y.size(0)

    avg_loss = sum(test_loss) / len(test_loss)
    accuracy = correct_predictions / total_samples

    print(f'Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}')



In [ ]:
# 1번 모델 훈련--LeNetCustum
train_loss1=[]
train_accuracy1=[]
for i in range(epoch):
  train_loss, train_accuracy = train(model1, optimizer1, RGB_32x32_4_train_loader, i)
  train_loss1.append(train_loss)
  train_accuracy1.append(train_accuracy)
  lr1.step()

In [ ]:
# model1 loss & accuracy 그래프
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss1)
plt.title('Train Loss (Model 1)')
plt.xlabel('epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracy1)
plt.title('Train Accuracy (Model 1)')
plt.xlabel('epoch')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
# 1번 모델 검증
test(model1,RGB_32x32_4_test_loader)


In [ ]:
# 2번 모델 훈련--AlexNetCustum
train_loss2=[]
train_accuracy2=[]
for i in range(epoch):
  train_loss, train_accuracy = train(model2, optimizer2, RGB_32x32_4_train_loader, i)
  train_loss2.append(train_loss)
  train_accuracy2.append(train_accuracy)
  lr1.step()

In [ ]:
# model2 loss & accuracy 그래프
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss2)
plt.title('Train Loss (Model 2)')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracy2)
plt.title('Train Accuracy (Model 2)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
# 2번 모델 검증
test(model2,RGB_32x32_4_test_loader)


In [ ]:
# 3번 모델 훈련--SimpleLeNet
train_loss3=[]
train_accuracy3=[]
for i in range(epoch):
  train_loss, train_accuracy = train(model3, optimizer3, RGB_32x32_4_train_loader, i)
  train_loss3.append(train_loss)
  train_accuracy3.append(train_accuracy)
  lr1.step()

In [ ]:
# model3 loss & accuracy 그래프
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss3)
plt.title('Train Loss (Model 3)')
plt.xlabel('epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracy3)
plt.title('Train Accuracy (Model 3)')
plt.xlabel('epoch')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
# 3번 모델 검증
test(model3,RGB_32x32_4_test_loader)
